In [1]:
# !python3 -m spacy download en_core_web_lg
# !pip install transformers
# !pip install tokenizer
# !pip install spacy

In [2]:
import numpy as np
import torch
import time
import datetime
import random
import pdb

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import * # for pad_sequence and whatnot
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, RandomSampler, SequentialSampler

from torch.utils import data
from torchvision import transforms

import matplotlib.pyplot as plt
import time

import json

from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

import logging
logging.basicConfig(level=logging.INFO)

from train_test import train, test
from preprocessing import PreprocessedData_wordlevel


cuda = torch.cuda.is_available()
print(cuda)
device = torch.device("cuda" if cuda else "cpu")

True


In [3]:
def make_graph(epochs, plots, name_long, name_short, save_name):
    colors = ['darkturquoise', 'mediumvioletred', 'beige', 'brown', 'chartreuse', 'chocolate', 
              'coral', 'crimson', 'fusia', 'goldenrod', 'green', 'indigo', 'lavender', 'lightgreen', 
              'orange', 'orchid']
    print('pp')
    BCK = (1, 1, 1)
    fig, ax = plt.subplots(random.randint(0, 10000000))
    ax.set_facecolor(BCK)
    print('oo')
    for idx, p in enumerate(plots):
        data = p[0]
        name = p[1]
        color = colors[idx]
        ax.plot(epochs, data, label=name, c=color)
        
    ax.set_title(name_long)
    ax.set_xlabel('Epochs')
    ax.set_ylabel(name_short)
    ax.legend()
    print('ii')
    plt.savefig(save_name, dpi=100)
    plt.clf()
    plt.close()
    plt.cla()
    print('hh')
    return
    
    
def run_epochs(model, train_dataloader, validation_dataloader, optimizer, scheduler, epochs):
    start_time = time.time()

    train_losses, train_accs = [], []
    test_losses, test_accs = [] , []
    
    for epoch in range(epochs):
        print('======== Epoch %d ========' % epoch)

        train_loss, train_acc = train(model, train_dataloader, optimizer, scheduler)
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        test_loss, test_acc = test(model, validation_dataloader)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        
    print("Total training took %.2f seconds" % (time.time()-start_time))
    
    return train_losses, train_accs, test_losses, test_accs
import pickle
def main():
    torch.cuda.empty_cache()
    MODEL_NAME = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    batches = [64, 32, 16, 8]
    max_lengths = [16, 32, 64]
    epoch_list = list(range(0, 4))
    
    for b in batches:
        train_losses = []
        train_accs = []
        test_losses = []
        test_accs = []
        ms = []
        for m in max_lengths:
            # For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
            batch_size = b
            max_length = m
            
            dataset = PreprocessedData_wordlevel(["./data/architecture_dz-cleaned-tagged.json",
                            "./data/design_dz-cleaned-tagged.json",
                           "./data/technology_dz-cleaned-tagged.json", ], 
                           ["./data/architecture_dz-cleaned.json", 
                            "./data/design_dz-cleaned.json",
                           "./data/technology_dz-cleaned.json"], tokenizer, max_length)
    
            train_dataset = dataset.train_dataset
            dev_dataset = dataset.dev_dataset
    
            # We'll take training samples in random order. 
            train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

            validation_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=batch_size)

            # Load BertForSequenceClassification, the pretrained BERT model with a single 
            # linear classification layer on top. 
            model = BertForSequenceClassification.from_pretrained(
                "bert-base-uncased",          # Use the 12-layer BERT model, with an uncased vocab.
                num_labels = 2,               # The number of output labels--2 for binary classification. 
                output_attentions = False,    # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )

            model.to(device)

            optimizer = optim.AdamW(model.parameters(), 
                          lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                          eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                        )

            # The BERT authors recommend between 2 and 4. 
            epochs = 4

            # Total number of training steps is [number of batches] x [number of epochs]. 
            # (Note that this is not the same as the number of training samples).
            total_steps = len(train_dataloader) * epochs

            # Create the learning rate scheduler.
            scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps = 0,
                                                        num_training_steps = total_steps)

            data = run_epochs(model, train_dataloader, validation_dataloader, optimizer, scheduler, epochs)
            
            train_l, train_a, test_l, test_a = data
            train_losses.append(train_l)
            train_accs.append(train_accs)
            test_losses.append(test_l)
            test_accs.append(test_a)
            ms.append(m)
            
            del model
            torch.cuda.empty_cache()
        
        # Make plot data
        plots = []
        for i in range(len(ms)):
            data = train_losses[i]
            name = 'Training, Max Length: %d' % ms[i]
            plots.append([data, name])
            data = test_losses[i]
            name = 'Testing, Max Length: %d' % ms[i]
            plots.append([data, name])
        name = './graphs/loss-b%d-unequal.pkl' % b
        with open(name, 'wb') as f:
            pickle.dump(plots, f)
#         make_graph(epoch_list, plots, 'Training and Testing loss, Batch Size: %d' % b, 
#                    'Loss', './graphs/loss-b%d-unequal.png' % b)
        plots = []
        for i in range(len(ms)):
            data = train_accs[i]
            name = 'Training, Max Length: %d' % ms[i]
            plots.append([data, name])
            data = test_accs[i]
            name = 'Testing, Max Length: %d' % ms[i]
            plots.append([data, name])
        name = './graphs/accs-b%d-unequal.pkl' % b
        with open(name, 'wb') as f:
            pickle.dump(plots, f)
    
main()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ubuntu/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_nor

12176 12176


INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ubuntu/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_nor

9,740 training samples
2,436 validation samples


INFO:transformers.modeling_utils:Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
INFO:transformers.modeling_utils:Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']


...

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/ubuntu/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_nor

9,740 training samples
2,436 validation samples


INFO:transformers.modeling_utils:Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
INFO:transformers.modeling_utils:Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']


======== Epoch 0 ========
======== Epoch 1 ========
======== Epoch 2 ========
======== Epoch 3 ========
Total training took 0.00 seconds
12176 12176


KeyboardInterrupt: 